# Lab 1: Sentence-BERT (SBERT) Training

### Fine-tuning Sentence-BERT (SBERT) & SBERT Embedding for applications
---


## Introduction
---

본 모듈에서는 문장 임베딩을 산출하는 Sentence-BERT 모델을 STS 데이터셋으로 파인튜닝해 봅니다.
SentenceTransformers 패키지를 사용하면 파인튜닝을 쉽게 수행할 수 있습니다. 다만, 현 시점에는 분산 훈련 기능 지원이 잘 되지 않으므로, 대용량 데이터셋으로 파인튜닝하는 니즈가 있다면 커스텀 훈련 코드를 직접 작성하셔야 합니다.

***[Note] SageMaker Studio Lab, SageMaker Studio, SageMaker 노트북 인스턴스, 또는 여러분의 로컬 머신에서 이 데모를 실행할 수 있습니다. SageMaker Studio Lab을 사용하는 경우 GPU를 활성화하세요.***

### References

- Hugging Face Tutorial: https://huggingface.co/docs/transformers/training
- Sentence-BERT paper: https://arxiv.org/abs/1908.10084
- SentenceTransformers: https://www.sbert.net


## 1. Setup Environments
---

### Import modules

In [1]:
# !pip install sentence_transformers datasets faiss-gpu progressbar

In [2]:
import os
import sys
import json
import logging
import argparse
import torch
import gzip
import csv
import math
import urllib
from torch import nn
import numpy as np
import pandas as pd
from tqdm import tqdm

from datetime import datetime
from datasets import load_dataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, SentencesDataset, LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from transformers.trainer_utils import get_last_checkpoint

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[LoggingHandler()]
)

logger = logging.getLogger(__name__)

### Argument parser

In [3]:
def parser_args(train_notebook=False):
    parser = argparse.ArgumentParser()

    # Default Setting
    parser.add_argument("--epochs", type=int, default=1)
    parser.add_argument("--seed", type=int, default=42)
    parser.add_argument("--train_batch_size", type=int, default=32)
    parser.add_argument("--eval_batch_size", type=int, default=32)
    parser.add_argument("--warmup_steps", type=int, default=100)
    parser.add_argument("--logging_steps", type=int, default=100)
    parser.add_argument("--learning_rate", type=str, default=5e-5)
    parser.add_argument("--disable_tqdm", type=bool, default=False)
    parser.add_argument("--fp16", type=bool, default=True)
    parser.add_argument("--tokenizer_id", type=str, default='sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
    parser.add_argument("--model_id", type=str, default='sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
    
    # SageMaker Container environment
    parser.add_argument("--output_data_dir", type=str, default=os.environ["SM_OUTPUT_DATA_DIR"])
    parser.add_argument("--model_dir", type=str, default=os.environ["SM_MODEL_DIR"])
    parser.add_argument("--n_gpus", type=str, default=os.environ["SM_NUM_GPUS"])
    parser.add_argument("--train_dir", type=str, default=os.environ["SM_CHANNEL_TRAIN"])
    parser.add_argument("--valid_dir", type=str, default=os.environ["SM_CHANNEL_VALID"])
    parser.add_argument("--test_dir", type=str, default=os.environ["SM_CHANNEL_TEST"])    
    parser.add_argument('--chkpt_dir', type=str, default='/opt/ml/checkpoints')     

    if train_notebook:
        args = parser.parse_args([])
    else:
        args = parser.parse_args()
    return args

In [4]:
train_dir = 'train'
valid_dir = 'valid'
test_dir = 'test'
!rm -rf {train_dir} {valid_dir} {test_dir} 
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True) 
os.makedirs(test_dir, exist_ok=True) 

### Load Arguments

주피터 노트북에서 곧바로 실행할 수 있도록 설정값들을 로드합니다. 물론 노트북 환경이 아닌 커맨드라인에서도 `cd scripts & python3 train.py` 커맨드로 훈련 스크립트를 실행할 수 있습니다.

In [5]:
chkpt_dir = 'chkpt'
model_dir = 'model'
output_data_dir = 'data'
num_gpus = torch.cuda.device_count()

!rm -rf {chkpt_dir} {model_dir} {output_data_dir} 

if os.environ.get('SM_CURRENT_HOST') is None:
    is_sm_container = False

    #src_dir = '/'.join(os.getcwd().split('/')[:-1])
    src_dir = os.getcwd()
    os.environ['SM_MODEL_DIR'] = f'{src_dir}/{model_dir}'
    os.environ['SM_OUTPUT_DATA_DIR'] = f'{src_dir}/{output_data_dir}'
    os.environ['SM_NUM_GPUS'] = str(num_gpus)
    os.environ['SM_CHANNEL_TRAIN'] = f'{src_dir}/{train_dir}'
    os.environ['SM_CHANNEL_VALID'] = f'{src_dir}/{valid_dir}'
    os.environ['SM_CHANNEL_TEST'] = f'{src_dir}/{test_dir}'
    
args = parser_args(train_notebook=True) 
args.chkpt_dir = chkpt_dir
logger.info("***** Arguments *****")
logger.info(''.join(f'{k}={v}\n' for k, v in vars(args).items()))

os.makedirs(args.chkpt_dir, exist_ok=True) 
os.makedirs(args.model_dir, exist_ok=True)
os.makedirs(args.output_data_dir, exist_ok=True) 

2022-07-20 08:42:26 - ***** Arguments *****
2022-07-20 08:42:26 - epochs=1
seed=42
train_batch_size=32
eval_batch_size=32
warmup_steps=100
logging_steps=100
learning_rate=5e-05
disable_tqdm=False
fp16=True
tokenizer_id=salti/bert-base-multilingual-cased-finetuned-squad
model_id=salti/bert-base-multilingual-cased-finetuned-squad
output_data_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/sentence-bert-finetuning/data
model_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/sentence-bert-finetuning/model
n_gpus=4
train_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/sentence-bert-finetuning/train
valid_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/sentence-bert-finetuning/valid
test_dir=/home/ec2-user/SageMaker/sm-kornlp-usecases/sentence-bert-finetuning/test
chkpt_dir=chkpt



<br>

## 2. Preparation
---
본 핸즈온에서 사용할 데이터셋은 KorSTS (https://github.com/kakaobrain/KorNLUDatasets) 와 KLUE-STS (https://github.com/KLUE-benchmark/KLUE) 입니다.
단일 데이터셋으로 훈련해도 무방하지만, 두 데이터셋을 모두 활용하여 훈련 시, 약간의 성능 향상이 있습니다.

### Training Tips
SBERT 훈련은 일반적으로 아래 3가지 방법들을 베이스라인으로 사용합니다.
1. NLI 데이터셋으로 훈련
2. STS 데이터셋으로 훈련
3. NLI 데이터셋으로 훈련 후 STS 데이터셋으로 파인튜닝

한국어 데이터의 경우, STS의 훈련 데이터가 상대적으로 적음에도 불구하고 NLI 기반 모델보다 예측 성능이 우수합니다. 따라서, 2번째 방법으로 진행합니다. <br>
다만, STS보다 조금 더 좋은 예측 성능을 원한다면 NLI 데이터로 먼저 훈련하고 STS 데이터셋으로 이어서 훈련하는 것을 권장합니다.

### KLUE-STS 데이터셋 다운로드 및 피쳐셋 생성
KLUE-STS 데이터셋을 허깅페이스 데이터셋 허브에서 다운로드 후, SBERT 훈련에 필요한 피쳐셋을 생성합니다.

In [6]:
logger.info("Read KLUE-STS train/dev dataset")
datasets = load_dataset("klue", "sts")

train_samples = []
dev_samples = []

for phase in ["train", "validation"]:
    examples = datasets[phase]

    for example in examples:
        score = float(example["labels"]["label"]) / 5.0  # 0.0 ~ 1.0 스케일로 유사도 정규화
        inp_example = InputExample(texts=[example["sentence1"], example["sentence2"]], label=score)

        if phase == "validation":
            dev_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

2022-07-20 08:42:30 - Read KLUE-STS train/dev dataset
2022-07-20 08:42:30 - Reusing dataset klue (/home/ec2-user/.cache/huggingface/datasets/klue/sts/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e)


  0%|          | 0/2 [00:00<?, ?it/s]


### KorSTS 데이터셋 다운로드 및 피쳐셋 생성
KorSTS 데이터셋은 허깅페이스에도 등록되어 있지만, 향후 여러분의 커스텀 데이터셋을 같이 사용하는 유즈케이스를 고려하여 GitHub의 데이터셋을 다운로드받아 사용하겠습니다. 

In [7]:
repo = 'https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorSTS'
urllib.request.urlretrieve(f'{repo}/sts-train.tsv', filename=f'{args.train_dir}/sts-train.tsv')
urllib.request.urlretrieve(f'{repo}/sts-dev.tsv', filename=f'{args.valid_dir}/sts-dev.tsv')
urllib.request.urlretrieve(f'{repo}/sts-test.tsv', filename=f'{args.test_dir}/sts-test.tsv')

# !wget https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorSTS/sts-train.tsv -O {train_dir}/sts-train.tsv
# !wget https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorSTS/sts-dev.tsv -O {valid_dir}/sts-dev.tsv
# !wget https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorSTS/sts-test.tsv -O {test_dir}/sts-test.tsv

('/home/ec2-user/SageMaker/sm-kornlp-usecases/sentence-bert-finetuning/test/sts-test.tsv',
 <http.client.HTTPMessage at 0x7fee52a7f970>)

In [8]:
logger.info("Read KorSTS train dataset")

with open(f'{args.train_dir}/sts-train.tsv', 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row["sentence1"] and row["sentence2"]:          
            score = float(row['score']) / 5.0  # Normalize score to range 0 ... 1
            inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=score)
            train_samples.append(inp_example)
            
logging.info("Read KorSTS dev dataset")            
with open(f'{args.valid_dir}/sts-dev.tsv', 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row["sentence1"] and row["sentence2"]:        
            score = float(row['score']) / 5.0  # Normalize score to range 0 ... 1
            inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=score)
            dev_samples.append(inp_example)            

2022-07-20 08:42:36 - Read KorSTS train dataset
2022-07-20 08:42:36 - Read KorSTS dev dataset


<br>

## 3. Training
---

### Training Preparation

### Model

In [9]:
model_name = 'sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens'

train_batch_size = args.train_batch_size
num_epochs = args.epochs
model_save_path = f'{args.model_dir}/training_sts_'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
logger.info(model_save_path)

# Use Huggingface/transformers model (like BERT, RoBERTa, XLNet, XLM-R) for mapping tokens to embeddings
word_embedding_model = models.Transformer(model_name)

2022-07-20 08:43:44 - /home/ec2-user/SageMaker/sm-kornlp-usecases/sentence-bert-finetuning/model/training_sts_sentence-transformers-xlm-r-100langs-bert-base-nli-stsb-mean-tokens-2022-07-20_08-43-44


문장 임베딩을 계산하기 위한 Pooler를 정의합니다. BERT로 분류 태스크를 수행할 때는 첫 번째 [CLS] 토큰의 출력 벡터를 임베딩 벡터로 사용하지만, SBERT에서는 BERT의 모든 토큰들의 출력 벡터들을 사용하여 임베딩 벡터를 계산합니다. 이 때 mean pooling이나 max pooling을 사용할 수 있으며, 본 예제에서는 mean pooling을 사용합니다.

In [11]:
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

2022-07-20 08:49:35 - Use pytorch device: cuda


모델 훈련 및 검증에 필요한 클래스 인스턴스를 생성합니다. 베이스라인으로 사용되는 검증 지표는 두 문장의 임베딩 벡터의 유사도를 산출하는 코사인 유사도입니다.

In [12]:
train_dataset = SentencesDataset(train_samples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) # 10% of train data for warm-up
logger.info("Warmup-steps: {}".format(warmup_steps))

2022-07-20 08:49:36 - Warmup-steps: 55


훈련을 수행합니다. 분산 훈련을 수행하지는 않지만, 데이터 볼륨이 크지 않으므로 수 분 내에 훈련이 완료됩니다.

### Start Training

In [13]:
# Train the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=num_epochs,
    evaluation_steps=int(len(train_dataloader)*0.5),
    warmup_steps=warmup_steps,
    output_path=model_save_path,
    use_amp=True
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/545 [00:00<?, ?it/s]

2022-07-20 08:50:25 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset in epoch 0 after 272 steps:
2022-07-20 08:50:28 - Cosine-Similarity :	Pearson: 0.8458	Spearman: 0.8462
2022-07-20 08:50:28 - Manhattan-Distance:	Pearson: 0.8333	Spearman: 0.8371
2022-07-20 08:50:28 - Euclidean-Distance:	Pearson: 0.8339	Spearman: 0.8380
2022-07-20 08:50:28 - Dot-Product-Similarity:	Pearson: 0.8095	Spearman: 0.8114
2022-07-20 08:50:28 - Save model to /home/ec2-user/SageMaker/sm-kornlp-usecases/sentence-bert-finetuning/model/training_sts_sentence-transformers-xlm-r-100langs-bert-base-nli-stsb-mean-tokens-2022-07-20_08-43-44
2022-07-20 08:51:12 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-dev dataset in epoch 0 after 544 steps:
2022-07-20 08:51:15 - Cosine-Similarity :	Pearson: 0.8511	Spearman: 0.8513
2022-07-20 08:51:15 - Manhattan-Distance:	Pearson: 0.8378	Spearman: 0.8416
2022-07-20 08:51:15 - Euclidean-Distance:	Pearson: 0.8383	Spearman: 0.8425
2022-07-20 08:51:15

<br>

## 4. Evaluation
---
훈련이 완료되었다면, 테스트 데이터셋으로 예측 성능을 볼 수 있는 지표들을 산출합니다.

In [14]:
test_samples = []
logger.info("Read KorSTS test dataset")            
with open(f'{args.test_dir}/sts-test.tsv', 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row["sentence1"] and row["sentence2"]:        
            score = float(row['score']) / 5.0  # Normalize score to range 0 ... 1
            inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=score)
            test_samples.append(inp_example)        

2022-07-20 08:51:47 - Read KorSTS test dataset


In [15]:
##############################################################################
# Load the stored model and evaluate its performance on STS benchmark dataset
##############################################################################

model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=model_save_path)

2022-07-20 08:51:48 - Load pretrained SentenceTransformer: /home/ec2-user/SageMaker/sm-kornlp-usecases/sentence-bert-finetuning/model/training_sts_sentence-transformers-xlm-r-100langs-bert-base-nli-stsb-mean-tokens-2022-07-20_08-43-44
2022-07-20 08:51:51 - Use pytorch device: cuda
2022-07-20 08:51:51 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-test dataset:
2022-07-20 08:51:53 - Cosine-Similarity :	Pearson: 0.8287	Spearman: 0.8310
2022-07-20 08:51:53 - Manhattan-Distance:	Pearson: 0.8242	Spearman: 0.8283
2022-07-20 08:51:53 - Euclidean-Distance:	Pearson: 0.8245	Spearman: 0.8287
2022-07-20 08:51:53 - Dot-Product-Similarity:	Pearson: 0.7619	Spearman: 0.7608


0.8309806357819561

<br>

## 5. Applications
---

In [16]:
import random
import time
from operator import itemgetter 

def get_faiss_index(emb, data, dim=768):
    import faiss
    n_gpus = torch.cuda.device_count()

    if n_gpus == 0:
        # Create the Inner Product Index
        index = faiss.IndexFlatIP(dim)
    else:
        flat_config = []
        res = [faiss.StandardGpuResources() for i in range(n_gpus)]
        for i in range(n_gpus):
            cfg = faiss.GpuIndexFlatConfig()
            cfg.useFloat16 = False
            cfg.device = i
            flat_config.append(cfg)

        index = faiss.GpuIndexFlatIP(res[0], dim, flat_config[0])

    index = faiss.IndexIDMap(index)
    index.add_with_ids(emb, np.array(range(0, len(data)))) 
    return index


def search(model, query, data, index, k=5, random_select=False, verbose=True):
    t = time.time()
    query_vector = model.encode(query)
    dists, top_k_inds = index.search(query_vector, k)
    if verbose:
        print('total time: {}'.format(time.time() - t))
    results = [itemgetter(*ind)(data) for ind in top_k_inds] 
    
    if random_select:
        return [random.choice(r) for r in results]
    else:
        return results

### Chatbot

챗봇은 크게 두 가지 형태로 개발합니다. 1) 생성 모델을 사용하여 해당 질문에 대한 창의적인 답변을 생성하거나, 2) 수많은 질문-답변 리스트들 중 질문에 부합하는 질문 후보들을 추린 다음 해당 후보에 적합한 답변을 찾는 방식이죠.
본 핸즈온은 2)의 방법으로 간단하게 챗봇 예시를 보여드립니다. 질문 텍스트를 입력으로 받으면, 해당 질문의 임베딩을 계산하여 질문 임베딩과 모든 질문 리스트의 임베딩을 비교하여 유사도가 가장 높은 질문 후보들을 찾고, 각 후보에 매칭되는 답변을 찾습니다.

코사인 유사도를 직접 계산할 수도 있지만, 페이스북에서 개발한 Faiss 라이브러리 (https://github.com/facebookresearch/faiss) 를 사용하면 훨씬 빠른 속도로 계산할 수 있습니다. Faiss는 Product 양자화 알고리즘을 GPU로 더욱 빠르게 구현한 라이브러리로, 정보 손실을 가급적 줄이면서 임베딩 벡터를 인덱싱합니다.

References
- Billion-scale similarity search with GPUs: https://arxiv.org/pdf/1702.08734.pdf
- Product Quantizers for k-NN Tutorial Part 1: https://mccormickml.com/2017/10/13/product-quantizer-tutorial-part-1
- Product Quantizers for k-NN Tutorial Part 1: http://mccormickml.com/2017/10/22/product-quantizer-tutorial-part-2

#### Preparing chatbot dataset

In [17]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", 
                           filename=f"{args.train_dir}/chatbot-train.csv")
chatbot_df = pd.read_csv(f'{args.train_dir}/chatbot-train.csv')
chatbot_df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


#### Embedding

In [18]:
chatbot_q_data = chatbot_df['Q'].tolist()
chatbot_a_data = chatbot_df['A'].tolist()
chatbot_emb = model.encode(chatbot_q_data, normalize_embeddings=True, batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/185 [00:00<?, ?it/s]

#### Indexing the dataset

In [19]:
chatbot_index = get_faiss_index(chatbot_emb, chatbot_q_data)

2022-07-20 08:52:14 - Loading faiss with AVX2 support.
2022-07-20 08:52:14 - Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2022-07-20 08:52:14 - Loading faiss.
2022-07-20 08:52:14 - Successfully loaded faiss.


#### Inference
샘플 질문들에 대한 추론을 수행합니다. 

In [36]:
query = ['커피 라떼 마시고 싶어']
search(model, query, chatbot_a_data, chatbot_index, random_select=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

total time: 0.03067159652709961


['좋은 시간 보내시길 바라요.']

In [37]:
query = ['너무 졸려', '놀고 싶어']
search(model, query, chatbot_a_data, chatbot_index, random_select=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

total time: 0.0308225154876709


['낮잠을 잠깐 자도 괜찮아요.', '같이 놀아요.']

### Semantic Search (News)


시멘틱(의미) 검색은 검색 쿼리가 키워드를 찾는 것뿐만 아니라, 검색에 사용되는 단어의 의도와 문맥적 의미를 파악하는 것을 목표로 합니다.
시멘틱 유사도 검색을 또한 상기 챗봇 예시와 마찬가지로, 해당 검색 쿼리를 입력하면, 검색 쿼리의 임베딩을 계산하여 모든 문서(예: 뉴스 제목/요약, 웹페이지 제목/요약) 리스트의 임베딩을 비교하여 가장 유사도가 높은 문서 후보들을 찾습니다.

References
- Billion-scale semantic similarity search with FAISS+SBERT: https://towardsdatascience.com/billion-scale-semantic-similarity-search-with-faiss-sbert-c845614962e2
- Korean Contemporary Corpus of Written Sentences: http://nlp.kookmin.ac.kr/kcc/

#### Preparing news dataset

In [38]:
import progressbar

class MyProgressBar():
    def __init__(self):
        self.pbar = None

    def __call__(self, block_num, block_size, total_size):
        if not self.pbar:
            self.pbar=progressbar.ProgressBar(maxval=total_size)
            self.pbar.start()

        downloaded = block_num * block_size
        if downloaded < total_size:
            self.pbar.update(downloaded)
        else:
            self.pbar.finish()

In [39]:
url = 'http://nlp.kookmin.ac.kr/kcc/KCCq28_Korean_sentences_EUCKR_v2.zip'
news_path = f'{args.train_dir}/KCCq28_Korean_sentences_EUCKR_v2.zip'
urllib.request.urlretrieve(url, news_path, MyProgressBar())

100% |########################################################################|


('/home/ec2-user/SageMaker/sm-kornlp-usecases/sentence-bert-finetuning/train/KCCq28_Korean_sentences_EUCKR_v2.zip',
 <http.client.HTTPMessage at 0x7f53a416d3a0>)

In [40]:
import zipfile
with zipfile.ZipFile(news_path, 'r') as zip_ref:
    zip_ref.extractall(train_dir)

In [41]:
!rm -rf {news_path}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [42]:
news_data = []
f = open(f'{args.train_dir}/KCCq28_Korean_sentences_EUCKR_v2.txt', 'rt', encoding='cp949')
lines = f.readlines()
for line in lines:
    line = line.strip()
    news_data.append(line)
f.close()

In [43]:
news_data = news_data[:10000] # For debug purpose

#### Embedding

In [44]:
news_emb = model.encode(news_data, normalize_embeddings=True, batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

#### Indexing the dataset

In [45]:
news_index = get_faiss_index(news_emb, news_data)

#### Inference
샘플 질문들에 대한 추론을 수행합니다. 

In [46]:
query =['아이스 라떼', '미세먼지']
search(model, query, news_data, news_index, k=7, random_select=False)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

total time: 0.031549692153930664


[('아이서플라이는 "아이폰4에 사용한 \'A4\'와 마찬가지로 \'다이 마크\'로 볼 때 삼성전자가 만든 것으로 보인다"고 했다.',
  '아이스크림을 서비스하고 있는 시공그룹의 박기석 회장은 "콘텐츠제공서비스 품질인증으로 아이스크림 서비스와 콘텐츠에 대해 신뢰도와 공신력을 인정받았다.',
  '엘리엇의 자회사인 블레이크 캐피탈과 포터 캐피탈은 홍보대행사를 통해 "삼성전자가 제시한 개략적인 주주가치 제고 방안이 향후 회사에 건설적인 첫 걸음이 될 것으로 보고 있다"고 밝혔다.',
  '검토 결과에 따라 결론을 내릴 것"이라고 말했다.',
  '조 대사는 "대미 의존도를 줄이려고 노력하고 있는 캐나다는 눈길을 아시아·태평양 지역으로 돌리고 있다"며 "특히 이 지역에서는 캐나다와 FTA를 맺은 국가가 없어 더욱 상징적 의미가 있다"고 말했다.',
  '박 대통령은 또 "적극적인 세일즈 외교 대통령으로 나서겠다"는 점을 분명히 했다.',
  '애플이 협력업체의 인력을 영입하는 이유에 대해 아이폰인 캐나다는 "애플이 GPU 자체 개발을 서두르기 위한 수순"이라고 설명했다.'),
 ('이회성 IPCC 의장은 "온실가스 배출량이 늘고 있다는 사실이 지구온난화의 과학적 근거"라고 말했다.',
  '다음은 일문 일답.파리기후협정의 의미는 무엇인가."2015년 12월 프랑스 파리에서 열린 기후변화협약 당사국총회에서 채택됐고, 지난해 11월 발효된 파리기후협정은 2020년부터 선진국뿐만 아니라 개발도상국도 의무적으로 온실가스 감축에 참여하도록 했다.',
  '미국에서 활동하고 있는 중국계 학자 유 사오카이는 이 논문에서 "도심의 고층빌딩 꼭대기에서 아래쪽으로 미세한 물방울을 뿌린다면 중국의 극심한 초미세먼지까지 낮출 수 있다"고 주장했다.',
  '그렇다면 대기 중 미세먼지 농도가 심한 날엔 어떻게 해야 할까.류연기 환경부 생활환경과장은 "대기 중 미세먼지 농도가 높은 날에 요리를 할 경우엔 우선 주방 환풍기를 사용해 환기하고, 요리 후엔 잠시 동안 창문을 열어 두는 것이 좋